In [7]:
from tensorflow.keras.layers import Add, Dense, Dropout, Layer, LayerNormalization, MultiHeadAttention
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, Callback
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError, MeanAbsoluteError
# from tensorflow_addons.metrics import RSquare
# from wandb.keras import WandbCallback

import math
import wandb
import pandas as pd
import tensorflow as tf
# import tensorflow_probability as tfp
# import tensorflow_addons as tfa
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import os
import matplotlib.pyplot as plt
# plt.style.use('dark_background')

from xc.tools.analysis import print_mats, decile_plot, AnalyseMatrix, load_overlap, _print_mats
from xclib.data import data_utils as du
import PIL.ImageOps as ImageOps
import scipy.sparse as sp
import numpy as np
import PIL.Image as Image
from io import BytesIO

import base64
import os
import pickle as p

from textwrap import wrap

ModuleNotFoundError: No module named 'xc'

In [8]:
# work_dir=f"{os.environ['HOME']}/scratch/XC"
# dset="G-LF-WikiTitles-1M"
# data_dir=f"{os.environ['HOME']}/data/{dset}"
# score_mat_dir=f"{work_dir}/score_mats/{dset}"
# os.makedirs(score_mat_dir, exist_ok=True)

In [9]:

_topk=5

raw_images=f"{data_dir}/images"
k = 5
A, B = 0.6, 2.6

tst_map = list(map(lambda x: x.strip(), open(f"{data_dir}/raw_data/test.raw.txt", "r").readlines()))
lbl_map = list(map(lambda x: x.strip(), open(f"{data_dir}/raw_data/label.raw.txt", "r").readlines()))

trn_y = du.read_sparse_file(f"{data_dir}/trn_X_Y.txt")
tst_y = du.read_sparse_file(f"{data_dir}/tst_X_Y.txt")

trn_imgs, lbl_imgs = None, None,
if os.path.exists(f"{data_dir}/images/label.img.bin.npz"):
    lbl_imgs = sp.load_npz(f"{data_dir}/images/label.img.bin.npz")
    trn_imgs = sp.load_npz(f"{data_dir}/images/train.img.bin.npz")
    tst_imgs = sp.load_npz(f"{data_dir}/images/test.img.bin.npz")
    imgs = open(f"{data_dir}/images/img.bin", "r")
filter_items = load_overlap(f"{data_dir}/filter_labels_test.txt")

NameError: name 'data_dir' is not defined

In [10]:
score_mats = {
    "ELIAS": f"{score_mat_dir}/ELIAS/score.npz",
    "NGAME": f"{score_mat_dir}/NGAME/score.npz"
}
score_mats = AnalyseMatrix(score_mats, _topk, filter_items)
sorted_mats = ["ELIAS", "NGAME"]
strict_mats = [-1, 2]

NameError: name 'score_mat_dir' is not defined

In [11]:
from xclib.data import data_utils

# Read file with features and labels (old format from XMLRepo)
features, tabels, num_samples, num_features, num_labels = data_utils.read_data('train.txt')

# Read sparse file (see docstring for more)
# header can be set to false (if required)
labels = data_utils.read_sparse_file('trn_X_Xf.txt', header=True)

# Write sparse file (with header)
data_utils.write_sparse_file(labels, "labels.txt")

ModuleNotFoundError: No module named 'xclib'

## Dataset Preparation
Ensure that you have:

amazon670k.train.txt – training data in LIBSVM format

amazon670k.test.txt – test data in LIBSVM format

In [12]:
# Clone the XML tools repo
!git clone https://github.com/xmc-aalto/xmltools.git
%cd xmltools
!pip install -r requirements.txt

# Convert from XML to LIBSVM (run only once)
!python -m xmltools.datasets.amazon_to_libsvm --dataset-path /path/to/downloaded/amazon/ --output-folder /content/


Cloning into 'xmltools'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'xmltools'
/content
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
/usr/bin/python3: Error while finding module specification for 'xmltools.datasets.amazon_to_libsvm' (ModuleNotFoundError: No module named 'xmltools')


In [13]:
/content/dismecpp/build/bin/train \
  /content/amazon670k.train.txt \
  amazon670k.model \
  --max-nr-labels 500000 \
  --normalize-instances \
  --threshold=0.01

SyntaxError: invalid syntax (ipython-input-13-883635568.py, line 1)

In [ ]:
/content/dismecpp/build/bin/predict \
  /content/amazon670k.test.txt \
  amazon670k.model \
  amazon670k_pred.txt \
  --augment-for-bias \
  --normalize-instances \
  --topk=5

In [ ]:
import numpy as np
import json

def parse_true_labels(file_path):
    labels = []
    with open(file_path) as f:
        for line in f:
            line = line.strip()
            if not line or line.startswith('#'):
                continue
            label_str = line.split(' ')[0]  # extract '1,2,3'
            label_ids = list(map(int, label_str.split(',')))
            labels.append(set(label_ids))
    return labels

def parse_pred_scores(file_path, top_k=5):
    predictions = []
    with open(file_path) as f:
        for line in f:
            line = line.strip()
            if not line:
                predictions.append([])
                continue
            parts = line.split()
            sorted_pairs = sorted([tuple(map(float, p.split(':')[::-1])) for p in parts], reverse=True)
            top_preds = [int(idx) for score, idx in sorted_pairs[:top_k]]
            predictions.append(top_preds)
    return predictions

def precision_at_k(true_labels, pred_labels, k):
    precisions = []
    for true, pred in zip(true_labels, pred_labels):
        pred_k = pred[:k]
        if not pred_k:
            precisions.append(0.0)
        else:
            precisions.append(len(set(pred_k) & true) / k)
    return np.mean(precisions)

def ndcg_at_k(true_labels, pred_labels, k):
    ndcgs = []
    for true, pred in zip(true_labels, pred_labels):
        dcg = 0.0
        for i, p in enumerate(pred[:k]):
            if p in true:
                dcg += 1.0 / np.log2(i + 2)
        ideal_dcg = sum(1.0 / np.log2(i + 2) for i in range(min(len(true), k)))
        ndcgs.append(dcg / ideal_dcg if ideal_dcg > 0 else 0.0)
    return np.mean(ndcgs)

def evaluate_all(true_file, pred_file, k_list=[1, 3, 5]):
    true_labels = parse_true_labels(true_file)
    results = {}
    for k in k_list:
        pred_labels = parse_pred_scores(pred_file, top_k=k)
        results[f"precision_at_{k}"] = precision_at_k(true_labels, pred_labels, k)
        results[f"ndcg_at_{k}"] = ndcg_at_k(true_labels, pred_labels, k)
    return results

# Example usage
true_file = "/content/amazon670k.test.txt"
pred_file = "/content/amazon670k_pred.txt"
k_list = [1, 3, 5]

results = evaluate_all(true_file, pred_file, k_list)

with open("amazon670k_metrics.json", "w") as f:
    json.dump(results, f, indent=4)

print(json.dumps(results, indent=4))
